In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
import requests
from dotenv import load_dotenv
load_dotenv()

@tool 
def get_coversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency factor between a given  currency and target currency"""
    url = f'https://v6.exchangerate-api.com/v6/18c8f9820ba7d32edff8e295/pair/{base_currency}/{target_currency}'

    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: float) -> float:
    """This function converts the base currency to target currency"""
    return base_currency_value * conversion_rate

In [18]:
# tool binding
llm = ChatGroq(model='qwen/qwen3-32b')
llm_with_tools = llm.bind_tools([get_coversion_factor, convert])

In [16]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage('What is the conversion factor between PKR and USD, and based on that can you convert 10 pkr to usd', additional_kwargs={}, response_metadata={})]
messages

[HumanMessage(content='What is the conversion factor between PKR and USD, and based on that can you convert 10 pkr to usd', additional_kwargs={}, response_metadata={})]

In [19]:
ai_message = llm_with_tools.invoke(messages)
ai_message

BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: attempted to call tool 'get_conversion_factor' which was not in request.tools", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<tool_call>\n{"name": "get_conversion_factor", "arguments": {"base_currency": "PKR", "target_currency": "USD"}}\n</tool_call>'}}